In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
import time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import gc
import glob
import joblib
import cupy as cp
import pandas as pd
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import os, time
from tqdm.notebook import tqdm

In [2]:
targets = ['reply','retweet','retweet_comment','like']

files = glob.glob('/raid/kaggle/2021/recsys/input/train_proc3/*')
print( len(files), files[1])

253 /raid/kaggle/2021/recsys/input/train_proc3/part-00243.parquet


In [3]:
start = time.time()
very_start = time.time()

In [4]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import cudf, cupy, time, rmm
cudf.__version__

'0.18.0'

In [5]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

import os
import time
import glob
import gc

import cupy as cp          # CuPy is an implementation of NumPy-compatible multi-dimensional array on GPU
import cudf                # cuDF is an implementation of Pandas-like Dataframe on GPU
import rmm                 # library for pre-allocating memory on GPU
import dask                # dask is an open-source library to nateively scale Python on multiple workers/nodes
import dask_cudf           # dask_cudf uses dask to scale cuDF dataframes on multiple workers/nodes

import numpy as np
import xgboost as xgb

# More dask / dask_cluster related libraries to scale NVTabular
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.distributed import wait
from dask.utils import parse_bytes
from dask.delayed import delayed

# NVTabular is the core library, we will use here for feature engineering/preprocessing on GPU
#import nvtabular as nvt
#from nvtabular.utils import device_mem_size
!rm -r dask-worker-space

rm: cannot remove 'dask-worker-space': No such file or directory


In [6]:
cluster = LocalCUDACluster(
    local_directory='/raid/kaggle/',
    CUDA_VISIBLE_DEVICES = '0,1,2,3',
    device_memory_limit = 0.50,
)
client = Client(cluster)

client

Client Scheduler: tcp://127.0.0.1:42951 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 270.39 GB


In [7]:
path = '/raid/kaggle/2021/recsys/input/train_proc3'
df = dask_cudf.read_parquet(f'{path}/*.parquet', npartitions=253)
df.columns

Index(['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'tw_len_token', 'text', 'tw_len_media',
       'tw_len_photo', 'tw_len_video', 'tw_len_gif', 'tw_len_quest',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
       'tw_original_user1', 'tw_original_user2', 'tw_rt_user0', 'tw_word0',
       'tw_word1', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
       'len_hashtags', 'len_links', 'len_domains', 'a_user_id32',
       'b_use

In [7]:
%%time
targets = ['reply','retweet','retweet_comment','like']

features = [
    'hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
    'language', 'timestamp', 'a_user_id', 'a_follower_count',
    'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
    'b_follower_count', 'b_following_count', 'b_is_verified',
    'b_account_creation', 'b_follows_a', 'reply', 'retweet',
    'retweet_comment', 'like', 'tw_len_token', 'tw_len_media',
    'tw_len_photo', 'tw_len_video', 'tw_len_gif', 'tw_len_quest',
    'tw_count_capital_words', 'tw_count_excl_quest_marks',
    'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
    'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
    'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
    'tw_original_user1', 'tw_original_user2', 'tw_rt_user0', 'tw_word0',
    'tw_word1', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
    'len_hashtags', 'len_links', 'len_domains',
    'decline'
]

CPU times: user 8 µs, sys: 3 µs, total: 11 µs
Wall time: 15.7 µs


In [9]:
path = '/raid/kaggle/2021/recsys/input/train_proc3'
df = dask_cudf.read_parquet(f'{path}/*.parquet', npartitions=253, columns=features )
_ = wait(df)
df.head()

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,tw_word1,tw_tweet,group,dt_day,dt_dow,dt_minute,len_hashtags,len_links,len_domains,decline
0,606344963,-7651340863584439249,1,0,0,1,1,1612712747,-1617640681192103974,526918,...,-518356270,1716595265,4,7,6,945,1,0,0,0
1,0,9050352748758673121,0,0,0,0,0,1614002023,2954260580541440200,3062,...,-1679042560,1238158712,3,22,0,833,0,0,0,0
2,0,-3385391863863081084,0,0,0,2,0,1612452111,174132517510076348,2298,...,-1270266736,2059407237,3,4,3,921,0,0,0,0
3,0,4878369566409755704,1,0,0,0,0,1612980390,160295098824211933,4813450,...,-708815550,332220573,4,10,2,1086,0,0,0,0
4,-1439756462,6645932351682048504,0,0,0,1,0,1612834174,7488217763927528828,2740151,...,990650098,-1865257506,4,9,1,89,1,0,0,0


In [9]:
a = df.shape
a[0].compute(),a[1]

(719714693, 52)

In [10]:
df.columns

Index(['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'tw_len_token', 'tw_len_media',
       'tw_len_photo', 'tw_len_video', 'tw_len_gif', 'tw_len_quest',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_word0', 'tw_word1',
       'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute', 'len_hashtags',
       'len_links', 'len_domains', 'decline'],
      dtype='object')

In [11]:
df['dt_day'].value_counts().compute()

16    36360477
4     36308997
22    35906781
9     35149455
17    34932862
5     34900064
23    34807671
18    34660036
11    34576269
15    34486248
10    34380091
21    34249787
19    34142425
12    34100380
14    33947305
8     33910950
7     33223126
24    32769889
6     32657891
20    32539778
13    31704211
Name: dt_day, dtype: int64

In [10]:
dt = df.loc[df.dt_day < 18]
dt.to_parquet( 'week12' )

dt = df.loc[df.dt_day >= 18]
dt.to_parquet( 'week3' )
gc.collect()

340

In [11]:
!mkdir week3-train
!mkdir week3-valid
files = glob.glob('week3/*.parquet')
for i, fn in tqdm(enumerate(files)):
    df = pd.read_parquet(fn)
    df = df.sample(frac=1.0)
    sz = int(df.shape[0]*0.20)
    df.iloc[:sz].to_parquet( 'week3-valid/part.'+str(i)+'.parquet' )
    df.iloc[sz:].to_parquet( 'week3-train/part.'+str(i)+'.parquet' )

0it [00:00, ?it/s]

In [9]:
trainfiles = glob.glob('week12/*.parquet') + glob.glob('week3-train/*.parquet')
len(trainfiles)

506

In [10]:
df = dask_cudf.read_parquet(trainfiles, npartitions=253, columns=features )
_ = wait(df)
df['decline'] = df['decline']//7.19
df.head()

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,tw_word1,tw_tweet,group,dt_day,dt_dow,dt_minute,len_hashtags,len_links,len_domains,decline
52449,0,9074165514232490788,0,0,0,0,1,1613969374,4696236357527432642,2340,...,-108567334,-605851715,3,22,0,289,0,0,0,0.0
2498775,0,-1553797528877028172,0,0,0,0,0,1614188507,-2087313690090317906,2468,...,-14393482,-1480179944,3,24,2,1061,0,0,0,47.0
859053,0,749897812826901074,3,0,0,1,3,1614038094,-361313288543262295,389,...,-859282570,1904083902,1,22,0,1434,0,0,0,0.0
1392212,0,1254509528808757918,0,0,0,1,1,1614071591,4080908988087942164,4117,...,-108567334,689777246,4,23,1,553,0,0,0,0.0
2229176,-296891246,5874348198213724069,0,0,0,1,1,1613734181,-7342811866416615614,72422,...,-108567334,776373637,4,19,4,689,1,0,0,0.0


In [23]:
!rm -r TEMAPS_LOCAL
!mkdir TEMAPS_LOCAL

In [24]:
for target in targets:
    feats = ['b_user_id', 'tweet_type']
    dt = df.groupby(feats)[target].agg(['sum','count'])
    fname = 'te_' + target + '-' + '-'.join(feats)
    dt.columns = ['sums_'+fname,'counts_'+fname]
    for f in dt.columns:
        dt[f] = dt[f].astype(np.int32)

    dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [25]:
%%time
dt0 = pd.read_parquet( 'TEMAPS_LOCAL/te_reply-b_user_id-tweet_type.parquet' )
dt1 = pd.read_parquet( 'TEMAPS_LOCAL/te_retweet-b_user_id-tweet_type.parquet' )
dt2 = pd.read_parquet( 'TEMAPS_LOCAL/te_retweet_comment-b_user_id-tweet_type.parquet' )
dt3 = pd.read_parquet( 'TEMAPS_LOCAL/te_like-b_user_id-tweet_type.parquet' )
dt0 = dt0.sort_index()
dt1 = dt1.sort_index()
dt2 = dt2.sort_index()
dt3 = dt3.sort_index()
dt0.shape, dt1.shape, dt2.shape, dt3.shape

CPU times: user 5min 56s, sys: 31.7 s, total: 6min 28s
Wall time: 5min 58s


((70970807, 2), (70970807, 2), (70970807, 2), (70970807, 2))

In [26]:
dt0['counts_te_reply-b_user_id-tweet_type'] = dt0['counts_te_reply-b_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_reply-b_user_id-tweet_type'] = dt0['sums_te_reply-b_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_retweet-b_user_id-tweet_type'] = dt1['sums_te_retweet-b_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_retweet_comment-b_user_id-tweet_type'] = dt2['sums_te_retweet_comment-b_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_like-b_user_id-tweet_type'] = dt3['sums_te_like-b_user_id-tweet_type'].astype(np.int16)

In [29]:
dt0.to_parquet('TEMAPS_LOCAL/b_user_id-tweet_type.parquet')

In [92]:
dt = pd.read_parquet('TEMAPS_LOCAL/b_user_id-tweet_type.parquet')
print(dt.shape)
dt.head()

(70968054, 5)


sums_te_reply-b_user_id-tweet_type  \
tweet_type b_user_id                                                  
0          -9223371402518308244                                   0   
           -9223370865369923345                                   3   
           -9223370184611917312                                   0   
           -9223369548747753919                                   0   
           -9223368068220830027                                   0   

                                 counts_te_reply-b_user_id-tweet_type  \
tweet_type b_user_id                                                    
0          -9223371402518308244                                     4   
           -9223370865369923345                                    53   
           -9223370184611917312                                     2   
           -9223369548747753919                                     9   
           -9223368068220830027                                     2   

                                 sums_te_retweet-b_user_id-tweet_type  \
tweet_type b_user_id                                                    
0          -9223371402518308244                                     2   
           -9223370865369923345                                     1   
           -9223370184611917312                                     0   
           -9223369548747753919                                     0   
           -9223368068220830027                                     0   

                                 sums_te_retweet_comment-b_user_id-tweet_type  \
tweet_type b_user_id                                                            
0          -9223371402518308244                                             0   
           -9223370865369923345                                             1   
           -9223370184611917312                                             0   
           -9223369548747753919                                             0   
           -9223368068220830027                                             0   

                                 sums_te_like-b_user_id-tweet_type  
tweet_type b_user_id                                                
0          -9223371402518308244                                  0  
           -9223370865369923345                                 22  
           -9223370184611917312                                  0  
           -9223369548747753919                                  5  
           -9223368068220830027                                  2

In [13]:
feats = ['b_user_id']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
dt.columns = ['sums_b_user_reply','sums_b_user_retweet','sums_b_user_retweet_comment','sums_b_user_like','counts_b_user_id']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/te-b_user_id.parquet')

In [31]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-b_user_id.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/b_user_id.parquet')
dt

,sums_b_user_reply,sums_b_user_retweet,sums_b_user_retweet_comment,sums_b_user_like,counts_b_user_id
b_user_id,,,,,
-9223371402518308244,0,2,0,0,5
-9223370865369923345,3,2,1,25,73
-9223370184611917312,0,0,0,0,3
-9223369548747753919,0,0,0,5,11
-9223369444486539912,0,0,0,0,1
...,...,...,...,...,...
9223370026930528603,0,0,0,1,2
9223370172135791871,0,0,0,1,1
9223370728254888202,0,0,0,2,5


In [50]:
for target in targets:
    feats = ['a_user_id', 'tweet_type']
    dt = df.groupby(feats)[target].agg(['sum','count'])
    fname = 'te_' + target + '-' + '-'.join(feats)
    dt.columns = ['sums_'+fname,'counts_'+fname]
    for f in dt.columns:
        dt[f] = dt[f].astype(np.int32)

    dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [51]:
dt0 = pd.read_parquet( 'TEMAPS_LOCAL/te_reply-a_user_id-tweet_type.parquet' )
dt1 = pd.read_parquet( 'TEMAPS_LOCAL/te_retweet-a_user_id-tweet_type.parquet' )
dt2 = pd.read_parquet( 'TEMAPS_LOCAL/te_retweet_comment-a_user_id-tweet_type.parquet' )
dt3 = pd.read_parquet( 'TEMAPS_LOCAL/te_like-a_user_id-tweet_type.parquet' )
dt0 = dt0.sort_index()
dt1 = dt1.sort_index()
dt2 = dt2.sort_index()
dt3 = dt3.sort_index()
dt0.shape, dt1.shape, dt2.shape, dt3.shape

((42009914, 2), (42009914, 2), (42009914, 2), (42009914, 2))

In [52]:
dt0['counts_te_reply-a_user_id-tweet_type'] = dt0['counts_te_reply-a_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_reply-a_user_id-tweet_type'] = dt0['sums_te_reply-a_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_retweet-a_user_id-tweet_type'] = dt1['sums_te_retweet-a_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_retweet_comment-a_user_id-tweet_type'] = dt2['sums_te_retweet_comment-a_user_id-tweet_type'].astype(np.int16)
dt0['sums_te_like-a_user_id-tweet_type'] = dt3['sums_te_like-a_user_id-tweet_type'].astype(np.int16)

In [56]:
dt0.to_parquet('TEMAPS_LOCAL/a_user_id-tweet_type.parquet')

In [48]:
feats = ['media','language','tweet_type','a_is_verified','b_is_verified','b_follows_a','tw_last_quest','decline','group']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['multi_reply','multi_retweet','multi_retweet_comment','multi_like','multi_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)
    
dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [49]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-media-language-tweet_type-a_is_verified-b_is_verified-b_follows_a-tw_last_quest-decline-group.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/media-language-tweet_type-a_is_verified-b_is_verified-b_follows_a-tw_last_quest-decline-group.parquet')
dt

multi_reply  \
media language tweet_type tw_last_quest b_is_verified group b_follows_a decline a_is_verified                
0     0        0          0             0             0     0           0.0     0                    32334   
                                                                                1                        0   
                                                                        1.0     0                        3   
                                                                        2.0     0                        3   
                                                                        3.0     0                        5   
...                                                                                                    ...   
12    2        0          0             0             3     0           0.0     0                        0   
                                                      4     0           0.0     0                        0   
      4        0          0             0             0     1           0.0     0                        0   
      9        0          0             0             4     0           0.0     0                        0   
13    10       0          0             0             1     1           0.0     0                        0   

                                                                                               multi_retweet  \
media language tweet_type tw_last_quest b_is_verified group b_follows_a decline a_is_verified                  
0     0        0          0             0             0     0           0.0     0                      38563   
                                                                                1                          4   
                                                                        1.0     0                         15   
                                                                        2.0     0                         11   
                                                                        3.0     0                          8   
...                                                                                                      ...   
12    2        0          0             0             3     0           0.0     0                          0   
                                                      4     0           0.0     0                          0   
      4        0          0             0             0     1           0.0     0                          1   
      9        0          0             0             4     0           0.0     0                          1   
13    10       0          0             0             1     1           0.0     0                          0   

                                                                                               multi_retweet_comment  \
media language tweet_type tw_last_quest b_is_verified group b_follows_a decline a_is_verified                          
0     0        0          0             0             0     0           0.0     0                               3765   
                                                                                1                                  0   
                                                                        1.0     0                                  1   
                                                                        2.0     0                                  2   
                                                                        3.0     0                                  2   
...                                                                                                              ...   
12    2        0          0             0             3     0           0.0     0                                  0   
                                                      4     0           0.0     0                                  0   
      4

In [34]:
feats = ['tw_word0', 'tweet_type']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['word_reply','word_retweet','word_retweet_comment','word_like','word_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [35]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-tw_word0-tweet_type.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/tw_word0-tweet_type.parquet')
print(dt.shape)
dt

(24797706, 5)


word_reply  word_retweet  word_retweet_comment  \
tw_word0    tweet_type                                                   
-2147483584 1                    0             0                     0   
-2147483189 0                    0             0                     0   
-2147483113 0                    0             0                     0   
-2147483019 0                    0             0                     0   
            1                    0             0                     0   
...                            ...           ...                   ...   
 2147483124 0                    1             0                     0   
 2147483271 0                    0             0                     0   
 2147483397 0                    0             0                     0   
 2147483444 0                    0             1                     0   
            1                    0             0                     0   

                        word_like  word_counts  
tw_word0    tweet_type                          
-2147483584 1                   0            1  
-2147483189 0                   1            1  
-2147483113 0                   0            1  
-2147483019 0                   3            3  
            1                   1            1  
...                           ...          ...  
 2147483124 0                   1            2  
 2147483271 0                   0            1  
 2147483397 0                   2            2  
 2147483444 0                   1            6  
            1                   0            1  

[24797706 rows x 5 columns]

In [39]:
feats = ['tw_original_user0', 'tweet_type']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['ouser0_reply','ouser0_retweet','ouser0_retweet_comment','ouser0_like','ouser0_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [40]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-tw_original_user0-tweet_type.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/tw_original_user0-tweet_type.parquet')
print(dt.shape)
dt

(8169419, 5)


ouser0_reply  ouser0_retweet  \
tweet_type tw_original_user0                                 
0          258                           0               0   
           2003                          0               0   
           3264                          3               1   
           5046                          0               0   
           5896                          0               0   
...                                    ...             ...   
2          4294942521                    1               0   
           4294947918                    0               0   
           4294955095                    0               0   
           4294961890                    0               0   
           4294966074                    0               1   

                              ouser0_retweet_comment  ouser0_like  \
tweet_type tw_original_user0                                        
0          258                                     0            1   
           2003                                    0            1   
           3264                                    0           18   
           5046                                    0            2   
           5896                                    0            1   
...                                              ...          ...   
2          4294942521                              0            5   
           4294947918                              0            1   
           4294955095                              0            1   
           4294961890                              0            3   
           4294966074                              0            0   

                              ouser0_counts  
tweet_type tw_original_user0                 
0          258                            1  
           2003                           1  
           3264                          22  
           5046                           2  
           5896                           2  
...                                     ...  
2          4294942521                    12  
           4294947918                     1  
           4294955095                     1  
           4294961890                    13  
           4294966074                    25  

[8169419 rows x 5 columns]

In [46]:
feats = ['tw_original_user1', 'tweet_type']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['ouser1_reply','ouser1_retweet','ouser1_retweet_comment','ouser1_like','ouser1_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [47]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-tw_original_user1-tweet_type.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/tw_original_user1-tweet_type.parquet')
print(dt.shape)
dt

(3051264, 5)


ouser1_reply  ouser1_retweet  \
tweet_type tw_original_user1                                 
0          3264                          0               1   
           11752                         0               0   
           12117                         0               0   
           12414                         0               0   
           15797                         0               0   
...                                    ...             ...   
2          4294891461                    0               0   
           4294907288                    0               0   
           4294925178                    0               0   
           4294942521                    0               0   
           4294961890                    0               0   

                              ouser1_retweet_comment  ouser1_like  \
tweet_type tw_original_user1                                        
0          3264                                    0           10   
           11752                                   0            0   
           12117                                   0            1   
           12414                                   0            2   
           15797                                   0            1   
...                                              ...          ...   
2          4294891461                              0            0   
           4294907288                              0            1   
           4294925178                              0            0   
           4294942521                              0            0   
           4294961890                              0            2   

                              ouser1_counts  
tweet_type tw_original_user1                 
0          3264                          11  
           11752                          1  
           12117                          1  
           12414                          2  
           15797                          1  
...                                     ...  
2          4294891461                     1  
           4294907288                     1  
           4294925178                     1  
           4294942521                     2  
           4294961890                     3  

[3051264 rows x 5 columns]

In [43]:
feats = ['tw_rt_user0' ]
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['rtuser0_reply','rtuser0_retweet','rtuser0_retweet_comment','rtuser0_like','rtuser0_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [45]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-tw_rt_user0.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/tw_rt_user0.parquet')
print(dt.shape)
dt

(10488117, 5)


,rtuser0_reply,rtuser0_retweet,rtuser0_retweet_comment,rtuser0_like,rtuser0_counts
tw_rt_user0,,,,,
164,0,0,0,0,1
421,0,0,0,0,1
499,0,0,0,0,3
701,0,0,0,1,1
1712,0,1,0,0,1
...,...,...,...,...,...
4294965572,1,5,0,59,121
4294965649,0,1,0,2,3
4294965968,0,3,0,4,25


In [24]:
df['a_follower_count'] = df['a_follower_count'] // 819
df['a_following_count'] = df['a_following_count'] // 712
df['b_follower_count'] = df['b_follower_count'] // 819
df['b_following_count'] = df['b_following_count'] // 712

In [25]:
(df['a_following_count']>100).mean().compute()

0.007275599758500796

In [29]:
feats = ['a_follower_count', 'a_following_count','b_follower_count', 'b_following_count', 'tweet_type', 'language', 'b_follows_a']
dt = df.groupby(feats)[targets].agg({
    "reply": ['sum'],
    "retweet": ['sum'],
    "retweet_comment": ['sum'],
    "like": ['sum','count'],
})
fname = 'te-' + '-'.join(feats)
dt.columns = ['follow_reply','follow_retweet','follow_retweet_comment','follow_like','follow_counts']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)

dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [30]:
dt = pd.read_parquet('TEMAPS_LOCAL/te-a_follower_count-a_following_count-b_follower_count-b_following_count-tweet_type-language-b_follows_a.parquet')
dt = dt.sort_index()
dt.to_parquet('TEMAPS_LOCAL/a_follower_count-a_following_count-b_follower_count-b_following_count-tweet_type-language-b_follows_a.parquet')
print(dt.shape)
dt

(10520707, 5)


follow_reply  \
a_follower_count tweet_type b_follower_count b_following_count b_follows_a a_following_count language                 
0                0          0                0                 0           0                 0               110780   
                                                                                             1                59751   
                                                                                             2                28868   
                                                                                             3                19995   
                                                                                             4                13960   
...                                                                                                             ...   
157802           2          73               1                 0           835               0                    0   
                            88               1                 0           835               0                    0   
                            178              202               0           835               0                    0   
                            195              0                 1           835               0                    0   
                            868              6                 0           835               0                    0   

                                                                                                       follow_retweet  \
a_follower_count tweet_type b_follower_count b_following_count b_follows_a a_following_count language                   
0                0          0                0                 0           0                 0                 211101   
                                                                                             1                  87558   
                                                                                             2                  59264   
                                                                                             3                  25680   
                                                                                             4                  53836   
...                                                                                                               ...   
157802           2          73               1                 0           835               0                      0   
                            88               1                 0           835               0                      1   
                            178              202               0           835               0                      0   
                            195              0                 1           835               0                      0   
                            868              6                 0           835               0                      0   

                                                                                                       follow_retweet_comment  \
a_follower_count tweet_type b_follower_count b_following_count b_follows_a a_following_count language                           
0                0          0                0                 0           0                 0                          18654   
                                                                                             1                           6290   
                                                                                             2                           4838   
                                                                                             3                           3780   
                                                                                             4                           3904   
...                                                                           

In [7]:
features = [
    'media', 'tweet_type', 'language', 
    'a_user_id32', 'a_is_verified', 
    'b_user_id32', 'b_is_verified', 'b_follows_a',
    'reply', 'retweet',  'retweet_comment', 'like',
    'tw_original_user0', 'tw_original_user1', 'tw_original_user2',
    'tw_rt_user0',
    'tw_word0', 'tw_word1', 
    'tw_tweet',
    'group', 
    'dt_dow',
    'decline'
]

In [8]:
path = '/raid/kaggle/2021/recsys/input/train_proc3'
df = dask_cudf.read_parquet(f'{path}/*.parquet', npartitions=500, columns=features )
_ = wait(df)
df.head()

,media,tweet_type,language,a_user_id32,a_is_verified,b_user_id32,b_is_verified,b_follows_a,reply,retweet,...,tw_original_user0,tw_original_user1,tw_original_user2,tw_rt_user0,tw_word0,tw_word1,tw_tweet,group,dt_dow,decline
0,1,1,1,-11345958,0,907661891,0,0,0,0,...,4186399962,4186399962,4186399962,4049978645,-708815550,-518356270,1716595265,4,6,0
1,0,0,0,2099028168,0,-28373041,0,1,0,0,...,953329835,4186399962,4186399962,4186399962,-108567334,-1679042560,1238158712,3,0,0
2,0,2,0,570972092,0,1903745470,0,1,0,0,...,4186399962,4186399962,4186399962,4186399962,550196236,-1270266736,2059407237,3,3,0
3,1,0,0,145178077,1,-810758208,0,0,0,0,...,4186399962,4186399962,4186399962,4186399962,590870355,-708815550,332220573,4,2,0
4,0,1,0,-877947524,1,366700355,0,0,0,0,...,4186399962,4186399962,4186399962,2116741992,97449781,990650098,-1865257506,4,1,0


In [9]:
df['positive'] = 1*(df['reply']>0)+2*(df['retweet']>0)+4*(df['retweet_comment']>0)+8*(df['like']>0)
df['positive'].value_counts().compute()

0     362514494
8     271013044
2      49907706
1      17248953
10     10855059
4       2748530
9       2709091
6       1404371
12       441959
3        325263
14       322451
11       142493
5         36510
7         26625
13        10243
15         7901
Name: positive, dtype: int64

In [13]:
feats = ['a_user_id32', 'b_user_id32']
for target in ['reply', 'retweet',  'retweet_comment', 'like']:
    dt = df[feats+[target,'positive']].loc[df.positive>0].groupby(feats)[target].agg('sum')
    fname = 'te-' + '-'.join(feats) + '_'+ target
    dt.columns = ['ab_'+target]
    for f in dt.columns:
        dt[f] = dt[f].astype(np.int32)
    dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [12]:
feats = ['a_user_id32', 'b_user_id32']
dt = df[feats+['positive']].loc[df.positive>0].groupby(feats)['positive'].agg('count')
fname = 'te-' + '-'.join(feats) + '_count'
dt.columns = ['ab_count']
for f in dt.columns:
    dt[f] = dt[f].astype(np.int32)
dt.to_parquet('TEMAPS_LOCAL/'+fname+'.parquet')

In [13]:
print( dt.head() )

                         ab_count
a_user_id32 b_user_id32          
 950500560   1364226896         1
-1848066873  834251964          1
 224772235  -1240161916         1
 1233438081 -663178678          1
-1272005296 -166308949          1
